In [ ]:
# Modules needed

In [ ]:
# Imports Needed

#### EDA

#### Preprocessing / Feature Engineering

#### Split Data 

#### Upload to Feature Store

#### Create Model Cards

#### Deploy model

#### Create Sagemaker endpoint

### Configure CloudWatch 

#### Create and upload baseline Metrics

#### Setup Monitoring Metrics